In [33]:
import json
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt

In [34]:
def check_avg(df_, season, team, stats = ['AST']):
    avg_stats = ['AVG_'+stat for stat in stats]
    df = df_.loc[(df_['SEASON_ID'] == season)& (df_['TEAM_ABBREVIATION'] == team)].sort_values('GAME_DATE')
    avg = df[stats].expanding().mean().shift(1)
    diff = df[avg_stats].values - avg[stats].values
    return(np.max(diff[1:]), np.min(diff[1:]))

In [35]:
STATS = ['PTS', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'DREB', 'OREB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
AVG_STATS = ['AVG_'+stat for stat in STATS]
OPP_AVG_STATS = ['OPP_'+stat for stat in AVG_STATS]

In [36]:
df = pd.read_csv('dat/1983-2017.csv', index_col=0).sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE'])
df = df.set_index('GAME_ID')
df['FG3A'] = df['FG3A'].fillna(0)
df.loc[df['FG3A']==0, ['FG3M', 'FG3_PCT']] = 0
df['FG2M'] = df['FGM'] - df['FG3M']
df['FG2A'] = df['FGA'] - df['FG3A']
df['FG2_PCT'] = df['FG2M'] / df['FG2A']
df['PLUS_MINUS'] = df['PTS'] - df['PTS_A']
df = df.reset_index()

In [37]:
for season in df.loc[df[STATS].isna().any(axis=1)]['SEASON_ID'].values:
    df = df.loc[df['SEASON_ID']!=season]

In [38]:
df[STATS].isna().any(axis=0)

PTS        False
FG2M       False
FG2A       False
FG2_PCT    False
FG3M       False
FG3A       False
FG3_PCT    False
FTM        False
FTA        False
FT_PCT     False
DREB       False
OREB       False
AST        False
STL        False
BLK        False
TOV        False
PF         False
dtype: bool

In [39]:
df.sample(10)

,GAME_ID,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year
58594,21000684,22010,1610612737,ATL,Atlanta Hawks,2011-01-28,ATL vs. NYK,W,239,111,42,82,0.512,7,19.0,0.368,20,23,0.870,13.0,33.0,46.0,22,7.0,4.0,12.0,20.0,9,102,35,63.0,0.555556,30.0,97.404255,95.361702,82,44.0,47,0.573169,26.938943,3.061057,3.061057,50.060915,-6.060915,6.060915,2010-11
41919,20300256,22003,1610612743,DEN,Denver Nuggets,2003-12-03,DEN @ GSW,W,239,117,40,67,0.597,7,11.0,0.636,30,36,0.833,6.0,25.0,31.0,23,7.0,7.0,14.0,24.0,8,109,33,56.0,0.589286,12.0,97.111111,92.833333,82,43.0,18,0.651729,11.731114,0.268886,0.268886,53.710626,-10.710626,10.710626,2003-04
10914,28800109,21988,1610612758,SAC,Sacramento Kings,1988-11-22,SAC vs. CHI,L,240,98,45,92,0.489,1,4.0,0.250,7,9,0.778,11.0,17.0,28.0,26,10.0,2.0,14.0,16.0,-16,114,44,88.0,0.500000,1.0,94.666667,111.555556,82,27.0,9,0.092501,0.832506,0.167494,0.167494,7.752552,19.247448,19.247448,1988-89
55828,20801037,22008,1610612763,MEM,Memphis Grizzlies,2009-03-21,MEM vs. BOS,L,241,87,33,84,0.393,5,11.0,0.455,16,21,0.762,17.0,26.0,43.0,17,8.0,2.0,15.0,22.0,-18,105,28,73.0,0.383562,17.0,93.289855,99.768116,82,24.0,69,0.282138,19.467542,-2.467542,2.467542,20.667798,3.332202,3.332202,2008-09
28529,29601103,21996,1610612754,IND,Indiana Pacers,1997-04-09,IND vs. CHI,L,240,80,29,74,0.392,4,18.0,0.222,18,20,0.900,14.0,30.0,44.0,18,7.0,9.0,17.0,18.0,-6,86,25,56.0,0.446429,37.0,94.973684,93.618421,82,39.0,76,0.549815,41.785952,-4.785952,4.785952,40.298891,-1.298891,1.298891,1996-97
22401,29300217,21993,1610612765,DET,Detroit Pistons,1993-12-08,DET vs. ATL,L,240,97,39,90,0.433,6,17.0,0.353,13,19,0.684,15.0,34.0,49.0,25,2.0,4.0,16.0,21.0,-8,105,33,73.0,0.452055,5.0,97.562500,102.250000,82,20.0,16,0.342367,5.477875,-0.477875,0.477875,27.596235,-7.596235,7.596235,1993-94
54410,20800678,22008,1610612746,LAC,Los Angeles Clippers,2009-01-28,LAC vs. CHI,L,240,75,27,75,0.360,6,20.0,0.300,15,21,0.714,8.0,25.0,33.0,19,10.0,8.0,13.0,19.0,-20,95,21,55.0,0.381818,10.0,92.866667,100.666667,82,19.0,45,0.245670,11.055149,-1.055149,1.055149,19.089789,-0.089789,0.089789,2008-09
21686,29300534,21993,1610612754,IND,Indiana Pacers,1994-01-27,IND @ DEN,L,240,106,44,87,0.506,1,4.0,0.250,17,19,0.895,13.0,21.0,34.0,19,7.0,5.0,17.0,30.0,-7,113,43,83.0,0.518072,16.0,98.256410,98.410256,82,47.0,39,0.494560,19.287822,-3.287822,3.287822,37.266060,9.733940,9.733940,1993-94
74204,21600977,22016,1610612753,ORL,Orlando Magic,2017-03-11,ORL vs. CLE,L,242,104,38,96,0.396,4,21.0,0.190,24,28,0.857,14.0,38.0,52.0,17,6.0,1.0,5.0,21.0,-12,116,34,75.0,0.453333,24.0,99.716418,106.074627,82,29.0,67,0.297379,19.924362,4.075638,4.075638,28.460678,0.539322,0.539322,2016-17
44682,20401118,22004,1610612747,LAL,Los Angeles Lakers,2005-04-07,LAL vs. HOU,L,240,100,33,87,0.379,10,29.0,0.345,24,28,0.857,16.0,28.0,44.0,21,6.0,3.0,10.0,19.0,-14,114,23,58.0,0.396552,33.0,98.080000,100.680000,82,34.0,75,0.410007,30.750514,2.249486,2.249486,35.870048,-1.870048,1.870048,2004-05


In [40]:
df[AVG_STATS] = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[STATS].expanding().mean().reset_index((0,1))[STATS]
df[AVG_STATS] = df.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS].shift(1)
df = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).set_index('GAME_ID', drop=True)

In [41]:
df.head(10)[['PTS', 'AVG_PTS', 'AST', 'AVG_AST']]

,PTS,AVG_PTS,AST,AVG_AST
GAME_ID,,,,
28500005,91,NaN,25,NaN
28500013,91,91.000000,14,25.000000
28500030,102,91.000000,21,19.500000
28500045,105,94.666667,22,20.000000
28500054,114,97.250000,25,20.500000
28500060,113,100.600000,18,21.400000
28500069,114,102.666667,17,20.833333
28500076,119,104.285714,25,20.285714
28500085,97,106.125000,26,20.875000


In [42]:
df['perc_complete'] = (df['GP'].values-1) / df['TOTAL_GAMES']

In [43]:
home = df.loc[~df['MATCHUP'].str.contains('@')].sort_index()
away = df.loc[df['MATCHUP'].str.contains('@')].sort_index()

In [44]:
home['HOME'] = 1
away['HOME'] = 0

In [45]:
opp_headers = ['OPP_'+stat for stat in STATS+AVG_STATS]

In [46]:
away_opp = away.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})
home_opp = home.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})

In [47]:
data = pd.concat([pd.concat([home, away_opp[opp_headers]], axis=1), pd.concat([away, home_opp[opp_headers]], axis=1)])

In [48]:
data['W'] = data['WL'].map({'W': 1, 'L': 0})

In [49]:
with open('dat/locations.json') as f:
    locations = json.loads(f.read())
data['LOCATION'] = data['TEAM_NAME'].map(locations)
data['LOCATION_W_PCT'] = data['LOCATION'].map(data.loc[data['HOME'] == 1].groupby('LOCATION')['W'].mean().to_dict())

In [50]:
data = data.reset_index()

In [51]:
data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).head()[['GAME_DATE', 'TEAM_ABBREVIATION', 'MATCHUP', 'PTS', 'AVG_PTS', 'OPP_PTS', 'OPP_AVG_PTS']]

,GAME_DATE,TEAM_ABBREVIATION,MATCHUP,PTS,AVG_PTS,OPP_PTS,OPP_AVG_PTS
21739,1985-10-25,ATL,ATL vs. WAS,91,NaN,100,NaN
58734,1985-10-26,ATL,ATL @ MIL,91,91.000000,117,116.000000
21764,1985-10-29,ATL,ATL vs. NYK,102,91.000000,87,89.000000
58766,1985-11-01,ATL,ATL @ BOS,105,94.666667,109,110.333333
21788,1985-11-02,ATL,ATL vs. PHL,114,97.250000,113,112.500000


*NOTE: OPP_AVG_PTS is not average points scored against, it is AVG_PTS for other team in matchup*

In [52]:
AVG_STATS_A = [stat+'_A' for stat in AVG_STATS]
OPP_STATS = ['OPP_'+stat for stat in STATS]

In [53]:
data[AVG_STATS_A] = data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[OPP_STATS].expanding().mean().reset_index((0,1))[OPP_STATS]
data[AVG_STATS_A] = data.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS_A].shift(1)

In [54]:
OPP_AVG_STATS_A = [stat+'_A' for stat in OPP_AVG_STATS]

In [55]:
data = data.set_index('GAME_ID')
home = data.loc[data['HOME']==1].sort_index()
away = data.loc[data['HOME']==0].sort_index()

In [56]:
home[OPP_AVG_STATS_A] = away[AVG_STATS_A]
away[OPP_AVG_STATS_A] = home[AVG_STATS_A]

In [57]:
new = pd.concat([home, away])

In [58]:
pts = new['AVG_PTS']
pts_a = new['AVG_PTS_A']
new['PYE_H'] = np.power(pts, 13.91)/(np.power(pts, 13.91) + np.power(pts_a, 13.91))

In [59]:
opp_pts = new['OPP_AVG_PTS']
opp_pts_a = new['OPP_AVG_PTS_A']
new['PYE_A'] = np.power(opp_pts, 13.91)/(np.power(opp_pts, 13.91) + np.power(opp_pts_a, 13.91))

In [60]:
phi_2017 = new.loc[(new['SEASON_ID']==22017) & (new['TEAM_ABBREVIATION']=='PHI')].sort_values('GAME_DATE')
phi_2017[['PTS', 'AVG_PTS', 'OPP_PTS', 'AVG_PTS_A']].head()

,PTS,AVG_PTS,OPP_PTS,AVG_PTS_A
GAME_ID,,,,
21700006,115,NaN,120,NaN
21700019,92,115.000000,102,120.000000
21700027,94,103.500000,128,111.000000
21700041,97,100.333333,86,116.666667
21700057,104,99.500000,105,109.000000


In [61]:
phi_2017[['MATCHUP', 'OPP_AVG_PTS', 'OPP_AVG_PTS_A']].head()

,MATCHUP,OPP_AVG_PTS,OPP_AVG_PTS_A
GAME_ID,,,
21700006,PHI @ WAS,NaN,NaN
21700019,PHI vs. BOS,99.5,105.0
21700027,PHI @ TOR,117.0,100.0
21700041,PHI @ DET,108.0,104.0
21700057,PHI vs. HOU,106.0,102.5


In [62]:
new.loc[21700091, ['TEAM_ABBREVIATION', 'MATCHUP', 'AVG_PTS', 'AVG_PTS_A', 'OPP_AVG_PTS', 'OPP_AVG_PTS_A']]

,TEAM_ABBREVIATION,MATCHUP,AVG_PTS,AVG_PTS_A,OPP_AVG_PTS,OPP_AVG_PTS_A
GAME_ID,,,,,,
21700091,GSW,GSW vs. DET,120.0,114.166667,104.5,99.500000
21700091,DET,DET @ GSW,104.5,99.500000,120.0,114.166667


In [63]:
new.to_csv('dat/expanding_avg_1983-2017.csv')

In [64]:
phi_2017[['PYE', 'PYE_H']]

,PYE,PYE_H
GAME_ID,,
21700006,0.356175,NaN
21700019,0.274258,0.356175
21700027,0.109297,0.274258
21700041,0.219522,0.109297
21700057,0.261008,0.219522
21700081,0.307052,0.261008
21700095,0.367577,0.307052
21700107,0.423549,0.367577
21700121,0.471439,0.423549
